# What is an agent?
- Large Language Models (LLMs) trained to perform causal language modeling can tackle a wide range of tasks, but they often struggle with basic tasks like logic, calculation, and search. When prompted in domains in which they do not perform well, they often fail to generate the answer we expect them to.

- One approach to overcome this weakness is to create an agent.

- **An agent is a system that uses an LLM as its engine, and it has access to functions called tools.**

- These tools are functions for performing a task, and they contain all necessary description for the agent to properly use them.

---

## The agent can be programmed to:

- devise a series of actions/tools and run them all at once like the CodeAgent for example
- plan and execute actions/tools one by one and wait for the outcome of each action before launching the next one like the ReactJsonAgent for example

---

## Types of agents

### Code agent
This agent has a planning step, then generates python code to execute all its actions at once. It natively handles different input and output types for its tools, thus it is the recommended choice for multimodal tasks.

### React agents
This is the go-to agent to solve reasoning tasks, since the ReAct framework makes it really efficient to think on the basis of its previous observations.

#### We implement two versions of ReactJsonAgent:

* ReactJsonAgent generates tool calls as a JSON in its output.
* ReactCodeAgent is a new type of ReactJsonAgent that generates its tool calls as blobs of code, which works really well for LLMs that have strong coding performance.

---

## How can I build an agent?

#### To initialize an agent, you need these arguments:

* an LLM to power your agent - the agent is not exactly the LLM, it’s more like the agent is a program that uses an LLM as its engine.
* a system prompt: what the LLM engine will be prompted with to generate its output
* a toolbox from which the agent pick tools to execute
* a parser to extract from the LLM output which tools are to call and with which arguments

Upon initialization of the agent system, the tool attributes are used to generate a tool description, then baked into the agent’s system_prompt to let it know which tools it can use and why.


# Examples

In [2]:
!pip install --quiet langchain langchain_openai langchain_community langchain_core wikipedia duckduckgo-search langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s e

In [3]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [4]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent # Agent Type: OPENAI Tools
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

In [5]:
llm = ChatOpenAI(temperature=0.5)

In [ ]:
messages = [
    SystemMessage(
        content="A user will input a year and you will get the IPL series winner for that year."
    ),
    HumanMessage(
        content="2023"
        ),
]

In [ ]:
result = llm.invoke(messages)

In [ ]:
result

AIMessage(content="I'm sorry, but I am unable to provide the IPL series winner for the year 2023 as the information is not available yet. The IPL series winners are determined at the end of each season, so we will have to wait until 2023 to find out the winner.", response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 31, 'total_tokens': 88}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-423c0326-652f-4955-822b-0a6863f64714-0', usage_metadata={'input_tokens': 31, 'output_tokens': 57, 'total_tokens': 88})

# LLM didnt gave the most recent information here, so we can use an agent and search the wikipedia or web and get the information.

In [ ]:
prompt = hub.pull("hwchase17/openai-tools-agent")

In [ ]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlacehold

In [ ]:
# Custom prompt: The agent prompt must have an agent_scratchpad key that is a MessagesPlaceholder. Intermediate agent actions and tool output messages will be passed in here.

from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [ ]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper=WikipediaAPIWrapper(
    top_k_results= 1,
    doc_content_chars_max = 500
)

In [ ]:
wiki_tools = WikipediaQueryRun(
    api_wrapper = api_wrapper
)

In [ ]:
tools = [wiki_tools] # Created a tool

https://api.python.langchain.com/en/latest/agents/langchain.agents.openai_tools.base.create_openai_tools_agent.html

In [ ]:
agent = create_openai_tools_agent(llm, tools, prompt)

In [ ]:
agent_executor = AgentExecutor(
    agent = agent,
    tools = tools
)

In [ ]:
agent_executor.invoke({"input": "what is a blackhole?"})

{'input': 'what is a blackhole?',
 'output': "A black hole is a region of spacetime where gravity is so strong that nothing, including light and other electromagnetic waves, can escape from it. According to Einstein's theory of general relativity, a sufficiently compact mass can deform spacetime to form a black hole. The boundary of no escape is called the event horizon. Objects that cross the event horizon of a black hole are influenced significantly by its gravity."}

# DuckDuckGo Agent

In [6]:
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [10]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults

api_wrapper = DuckDuckGoSearchAPIWrapper(region="de-de", time="d", max_results=2)

In [11]:
duck_duck_tools = DuckDuckGoSearchResults(
    api_wrapper=api_wrapper,
    source="news"
    )

In [12]:
tools = [duck_duck_tools] # Created a tool

https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.duckduckgo_search.DuckDuckGoSearchAPIWrapper.html

In [13]:
agent = create_openai_tools_agent(llm, tools, prompt)

In [14]:
agent_executor = AgentExecutor(
    agent = agent,
    tools = tools
)

In [15]:
agent_executor.invoke({"input": "what time is NBA final today?"})

{'input': 'what time is NBA final today?',
 'output': 'The NBA Finals start tonight at 8:30 p.m.'}

In [16]:
agent_executor.invoke({"input": "Who won the cricket worldcup match today?"})

{'input': 'Who won the cricket worldcup match today?',
 'output': "The USA cricket team won the match today in the T20 World Cup against Pakistan in a thrilling 'super over' match."}